In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [31]:
train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")
train_dataset.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [32]:
print(train_dataset.isnull().sum().sort_values())
print("\n\n")
print(train_dataset.isna().sum().sort_values())


print(test_dataset.isnull().sum().sort_values())
print("\n\n")
print(test_dataset.isna().sum().sort_values())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687
dtype: int64



PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
Fare             1
Age             86
Cabin          327
dtype: int64



PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
Fare             1
Age             86
Cabin          327
dtype: int64


In [33]:
train_dataset_encoded = pd.get_dummies(data=train_dataset, columns=["Sex", "Pclass", "Embarked"], dtype=int)
test_dataset_encoded = pd.get_dummies(data=test_dataset, columns=["Sex", "Pclass", "Embarked"], dtype=int)

In [34]:
train_dataset_encoded.drop(columns=["PassengerId", "Ticket", "Name", "Cabin"], axis=1, inplace=True)
test_dataset_encoded.drop(columns=["PassengerId", "Ticket", "Name", "Cabin"], axis=1, inplace=True)

In [35]:
print(len(train_dataset) * 0.22)
print(len(train_dataset) - (len(train_dataset) * 0.22))
print(len(test_dataset))

196.02
694.98
418


In [36]:
imputer = IterativeImputer(estimator=RandomForestRegressor(), random_state=10)
train_dataset_encoded["Age"] = imputer.fit_transform(train_dataset_encoded[["Age"]])
train_dataset_encoded["Age"] = imputer.fit_transform(train_dataset_encoded[["Age"]])

In [37]:
X_train = train_dataset_encoded.drop(columns="Survived", axis=1)
y_train = train_dataset_encoded["Survived"]

In [38]:
X_train

Age  SibSp  Parch     Fare  Sex_female  Sex_male  Pclass_1  \
0    22.000000      1      0   7.2500           0         1         0   
1    38.000000      1      0  71.2833           1         0         1   
2    26.000000      0      0   7.9250           1         0         0   
3    35.000000      1      0  53.1000           1         0         1   
4    35.000000      0      0   8.0500           0         1         0   
..         ...    ...    ...      ...         ...       ...       ...   
886  27.000000      0      0  13.0000           0         1         0   
887  19.000000      0      0  30.0000           1         0         1   
888  29.699118      1      2  23.4500           1         0         0   
889  26.000000      0      0  30.0000           0         1         1   
890  32.000000      0      0   7.7500           0         1         0   

     Pclass_2  Pclass_3  Embarked_C  Embarked_Q  Embarked_S  
0           0         1           0           0           1  
1           0         0           1           0           0  
2           0         1           0           0           1  
3           0         0           0           0           1  
4           0         1           0           0           1  
..        ...       ...         ...         ...         ...  
886         1         0           0           0           1  
887         0         0           0           0           1  
888         0         1           0           0           1  
889         0         0           1           0           0  
890         0         1           0           1           0  

[891 rows x 12 columns]

In [39]:
train_dataset

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ...    ...   
886                              Montvila, Rev. Juozas    male  27.0      0   
887                       Graham, Miss. Margaret Edith  female  19.0      0   
888           Johnston, Miss. Catherine Helen "Carrie"  female   NaN      1   
889                              Behr, Mr. Karl Howell    male  26.0      0   
890                                Dooley, Mr. Patrick    male  32.0      0   

     Parch            Ticket     Fare Cabin Embarked  
0        0         A/5 21171   7.2500   NaN        S  
1        0          PC 17599  71.2833   C85        C  
2        0  STON/O2. 3101282   7.9250   NaN        S  
3        0            113803  53.1000  C123        S  
4        0            373450   8.0500   NaN        S  
..     ...               ...      ...   ...      ...  
886      0            211536  13.0000   NaN        S  
887      0            112053  30.0000   B42        S  
888      2        W./C. 6607  23.4500   NaN        S  
889      0            111369  30.0000  C148        C  
890      0            370376   7.7500   NaN        Q  

[891 rows x 12 columns]